# Load Depenency

In [1]:
#Cleaned up Crypto Example
from os import system
import pandas as pd
import requests
import datetime as dt
# from pytrials.client import ClinicalTrials
import json
import ipywidgets as widgets
import numpy as np
np.random.seed(10031975)
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rcParams, cycler
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator

In [2]:
#does some error handling


import sys
import csv
maxInt = sys.maxsize

while True:
    # decrease the maxInt value by factor 10 
    # as long as the OverflowError occurs.

    try:
        csv.field_size_limit(maxInt)
        break
    except OverflowError:
        maxInt = int(maxInt/10)

In [3]:
from pytrials.utils import json_handler, csv_handler


class ClinicalTrials:
    """ClinicalTrials API client

    Provides functions to easily access the ClinicalTrials.gov API
    (https://clinicaltrials.gov/api/)
    in Python.

    Attributes:
        study_fields: List of all study fields you can use in your query.
        api_info: Tuple containing the API version number and the last
        time the database was updated.
    """

    _BASE_URL = "https://clinicaltrials.gov/api/"
    _INFO = "info/"
    _QUERY = "query/"
    _JSON = "fmt=json"
    _CSV = "fmt=csv"

    def __init__(self):
        self.api_info = self.__api_info()

    @property
    def study_fields(self):
        fields_list = json_handler(
            f"{self._BASE_URL}{self._INFO}study_fields_list?{self._JSON}"
        )
        return fields_list["StudyFields"]["Fields"]

    def __api_info(self):
        """Returns information about the API"""
        last_updated = json_handler(
            f"{self._BASE_URL}{self._INFO}data_vrs?{self._JSON}"
        )["DataVrs"]
        api_version = json_handler(f"{self._BASE_URL}{self._INFO}api_vrs?{self._JSON}")[
            "APIVrs"
        ]

        return api_version, last_updated

    def get_full_studies(self, search_expr, max_studies=50):
        """Returns all content for a maximum of 100 study records.

        Retrieves information from the full studies endpoint, which gets all study fields.
        This endpoint can only output JSON (Or not-supported XML) format and does not allow
        requests for more than 100 studies at once.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.
            max_studies (int): An integer indicating the maximum number of studies to return.
                Defaults to 50.

        Returns:
            dict: Object containing the information queried with the search expression.

        Raises:
            ValueError: The number of studies can only be between 1 and 100
        """
        if max_studies > 100 or max_studies < 1:
            raise ValueError("The number of studies can only be between 1 and 100")

        req = f"full_studies?expr={search_expr}&max_rnk={max_studies}&{self._JSON}"

        full_studies = json_handler(f"{self._BASE_URL}{self._QUERY}{req}")

        return full_studies

    def get_study_fields(self, search_expr, fields, max_studies=50, min_rnk=1,fmt="csv"):
        """Returns study content for specified fields

        Retrieves information from the study fields endpoint, which acquires specified information
        from a large (max 1000) studies. To see a list of all possible fields, check the class'
        study_fields attribute.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.
            fields (list(str)): A list containing the desired information fields.
            max_studies (int): An integer indicating the maximum number of studies to return.
                Defaults to 50.
            fmt (str): A string indicating the output format, csv or json. Defaults to csv.

        Returns:
            Either a dict, if fmt='json', or a list of records (e.g. a list of lists), if fmt='csv.
            Both containing the maximum number of study fields queried using the specified search expression.

        Raises:
            ValueError: The number of studies can only be between 1 and 1000
            ValueError: One of the fields is not valid! Check the study_fields attribute
                for a list of valid ones.
            ValueError: Format argument has to be either 'csv' or 'json'
        """
        if max_studies > 1000 or max_studies < 1:
            raise ValueError("The number of studies can only be between 1 and 1000")
        elif not set(fields).issubset(self.study_fields):
            raise ValueError(
                "One of the fields is not valid! Check the study_fields attribute for a list of valid ones."
            )
        else:
            concat_fields = ",".join(fields)
            # req = f"study_fields?expr={search_expr}&max_rnk={max_studies}&fields={concat_fields}"
            req = f"study_fields?expr={search_expr}&min_rnk={min_rnk}&max_rnk={max_studies+min_rnk-1}&fields={concat_fields}"
            if fmt == "csv":
                url = f"{self._BASE_URL}{self._QUERY}{req}&{self._CSV}"
                return csv_handler(url)

            elif fmt == "json":
                url = f"{self._BASE_URL}{self._QUERY}{req}&{self._JSON}"
                return json_handler(url)

            else:
                raise ValueError("Format argument has to be either 'csv' or 'json'")

    def get_study_count(self, search_expr):
        """Returns study count for specified search expression

        Retrieves the count of studies matching the text entered in search_expr.

        Args:
            search_expr (str): A string containing a search expression as specified by
                `their documentation <https://clinicaltrials.gov/api/gui/ref/syntax#searchExpr>`_.

        Returns:
            An integer

        Raises:
            ValueError: The search expression cannot be blank.
        """
        if not set(search_expr):
            raise ValueError("The search expression cannot be blank.")
        else:
            req = f"study_fields?expr={search_expr}&max_rnk=1&fields=NCTId"
            url = f"{self._BASE_URL}{self._QUERY}{req}&{self._JSON}"
            returned_data = json_handler(url)
            study_count = returned_data["StudyFieldsResponse"]["NStudiesFound"]
            return study_count

    def __repr__(self):
        return f"ClinicalTrials.gov client v{self.api_info[0]}, database last updated {self.api_info[1]}"

# Load Data

https://clinicaltrials.gov/api/gui/ref/crosswalks

Disease 2 search for

This is for dymanic input of diease searching

In [4]:
# disease=input()
# print(disease)

In [5]:
# disease = 'parkinson'

#disease = 'depression'

# disease = 'diabetes'

disease = 'tacs'



hit mesh some day

Pull ONE

#Explore Data

In [6]:
#change this out to NAME

ct = ClinicalTrials()
infodf= (ct.get_full_studies(search_expr=disease, max_studies=5 ))


In [7]:
type(infodf)

dict

In [8]:
number = print(infodf['FullStudiesResponse']['NStudiesFound'])

1268


In [9]:
count = ct.get_study_count(search_expr=disease)


In [10]:
# count = 3000
start = 0
end = count
iter = 999 #number of studies to get at a time ( limited to 1000 )

from random import seed
from random import random



temp = pd.DataFrame()
temp2 = pd.DataFrame()
new_df = pd.DataFrame()


for i in range(1, count, iter):
    # print(temp)
    temp = ct.get_study_fields(
        search_expr=disease,
        fields=["NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType",
    "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase" ,"StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","BaselineMeasureTitle","BaselineMeasureDescription","BaselineMeasurementValue","BriefSummary"],
        max_studies=iter,
        min_rnk=i,
        fmt="csv",)
    # print(temp2[0])
    temp2 =  pd.DataFrame.from_records(temp[1:])
    new_df = pd.concat([new_df, temp2])
 


In [11]:
# count = 3000
start = 0
end = count
iter = 999 #number of studies to get at a time ( limited to 1000 )

from random import seed
from random import random



temp = pd.DataFrame()
temp2 = pd.DataFrame()
new_df = pd.DataFrame()


for i in range(1, count, iter):
    # print(temp)
    temp = ct.get_study_fields(
        search_expr=disease,
        fields=["NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType",
    "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase" ,"StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","BaselineMeasureTitle","BaselineMeasureDescription","BaselineMeasurementValue","BriefSummary"],
        max_studies=iter,
        min_rnk=i,
        fmt="csv",)
    # print(temp2[0])
    temp2 =  pd.DataFrame.from_records(temp[1:])
    new_df = pd.concat([new_df, temp2])

In [12]:
new_df.columns = ["Index","NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType",
    "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase" ,"StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","BaselineMeasureTitle","BaselineMeasureDescription","BaselineMeasurementValue","BriefSummary"]

'NStudiesFound': 3656,

In [13]:
# new_df
new_df.shape

(1268, 20)

In [14]:
new_df.head

<bound method NDFrame.head of     Index        NCTId                                          Condition  \
0       1  NCT00890968                                    Hand Dermatoses   
1       2  NCT04727151                         HER2-Positive Solid Tumors   
2       3  NCT03378076                      Bilateral Knee Osteoarthritis   
3       4  NCT00077142                                       Liver Cancer   
4       5  NCT04515433                                  Alzheimer Disease   
..    ...          ...                                                ...   
264  1264  NCT05111743             Age-related Macular Degeneration (AMD)   
265  1265  NCT03830905       Influenza|Acute Respiratory Viral Infections   
266  1266  NCT03540485  Sclerosis, Multiple|Autoimmune Diseases of the...   
267  1267  NCT03564652                                     Undernutrition   
268  1268  NCT03894683       Heart Failure With Reduced Ejection Fraction   

                                         Offi

In [15]:
# temp = pd.DataFrame.from_records(api_pull_2[1:], columns=api_pull_2[0])

In [16]:
# print(temp.shape)

In [17]:
# count = 3000
start = 0
end = count
iter = 999 #number of studies to get at a time ( limited to 1000 )

from random import seed
from random import random



temp = pd.DataFrame()
temp2 = pd.DataFrame()
new_df2 = pd.DataFrame()


for i in range(1, count, iter):
    # print(temp)
    temp = ct.get_study_fields(
        search_expr=disease,
        fields=["NCTId","IsFDARegulatedDrug","IsFDARegulatedDevice", "IsUnapprovedDevice", "PrimaryOutcomeMeasure", "PrimaryOutcomeDescription","PrimaryOutcomeTimeFrame", "SecondaryOutcomeMeasure","SecondaryOutcomeDescription", "SecondaryOutcomeTimeFrame","OtherOutcomeMeasure"
,"OtherOutcomeDescription","OtherOutcomeTimeFrame","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity", "LocationState" , "LocationFacility"],
        max_studies=iter,
        min_rnk=i,
        fmt="csv",)
    # print(temp2[0])
    temp2 =  pd.DataFrame.from_records(temp[1:])
    new_df2 = pd.concat([new_df2, temp2])
 


In [18]:
new_df2.columns = ["index", "NCTId","IsFDARegulatedDrug","IsFDARegulatedDevice", "IsUnapprovedDevice", "PrimaryOutcomeMeasure", "PrimaryOutcomeDescription","PrimaryOutcomeTimeFrame", "SecondaryOutcomeMeasure","SecondaryOutcomeDescription", "SecondaryOutcomeTimeFrame","OtherOutcomeMeasure"
,"OtherOutcomeDescription","OtherOutcomeTimeFrame","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity", "LocationState" , "LocationFacility"]

In [19]:
new_df2.shape

(1268, 21)

In [20]:
new_df2

,index,NCTId,IsFDARegulatedDrug,IsFDARegulatedDevice,IsUnapprovedDevice,PrimaryOutcomeMeasure,PrimaryOutcomeDescription,PrimaryOutcomeTimeFrame,SecondaryOutcomeMeasure,SecondaryOutcomeDescription,...,OtherOutcomeMeasure,OtherOutcomeDescription,OtherOutcomeTimeFrame,EligibilityCriteria,StudyPopulation,HealthyVolunteers,ReferencePMID,LocationCity,LocationState,LocationFacility
0,1,NCT00890968,,,,Response as assessed by Investigator Global As...,,"Baseline, Week 1, Week 2, Week 4",Subject's Global Impression of Change (SGIC)|I...,,...,,,,Inclusion Criteria:||Clinical diagnosis of sta...,,No,,San Diego|Clinton Township|Austin,California|Michigan|Texas,Therapeutics Clinical Research|Michigan Center...
1,2,NCT04727151,Yes,No,,Incidence of Treatment-Emergent Adverse Events,Assessed by Common Terminology Criteria for Ad...,24 Months,Determine MTD or RP2D|Determine expansion of T...,Assessed by incidence of Dose-Limiting Toxicit...,...,,,,Inclusion Criteria:||Age ≥ 18 years.|A recent ...,,No,,Chicago|Boston|Newark|Houston|Toronto,Illinois|Massachusetts|New Jersey|Texas|Ontario,University of Chicago|Dana Farber Cancer Insti...
2,3,NCT03378076,Yes,No,,Measure the Concentration of Triamcinolone Ace...,Plasma drug concentrations (pg/mL) by Time Poi...,43 days|43 days,,,...,,,,Inclusion Criteria:||Written consent to partic...,,No,31662801,Torrance|Rochester|Duncansville,California|New York|Pennsylvania,LA Biomed at Harbor-UCLA Medical Center|Roches...
3,4,NCT00077142,,,,Maximum Tolerated Dose (MTD) of TAC-101,,60 Days,,,...,,,,DISEASE CHARACTERISTICS:||Histologically or cy...,,No,18504614,Houston,Texas,MD Anderson Cancer Center at University of Texas
4,5,NCT04515433,No,No,,Changes in Rey Auditory Verbal Learning Test s...,Participants are given a list of 15 unrelated ...,Baseline (immediately before the intervention)...,Change in SAI measurements,By using transcranial magnetic stimulation (TM...,...,,,,Inclusion Criteria:||- Mild Cognitive Impairme...,,No,19400723|27929004|31076275|30879788|23785325,Brescia,BS,AO Spedali Civili
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,1264,NCT05111743,No,No,,Number of patient eyes with an Intraocular Inf...,To assess IOI events observed after starting t...,Up to 6 months post brolucizumab injection,Age|Gender information|Number of patients at v...,Age information was reported|Gender informatio...,...,,,,Inclusion criteria:||≥1 Healthcare Common Proc...,Patients with wet AMD who initiated brolucizum...,No,,East Hanover,New Jersey,Novartis Investigative Site
265,1265,NCT03830905,No,No,,Time to sustained improvement in clinical symp...,The time before the onset of sustained improve...,From the moment of the first dose up to Day 14,The time before the onset of persistent improv...,The time of the first assessment of 3 consecut...,...,The average concentration of MxA protein in th...,"The average concentration of 6Ckine, BCA-1, CT...",From Day 1 up to Day 6|From Day 1 up to Day 6|...,Inclusion Criteria:||Both sexes aged 18 years ...,,No,,Arkhangelsk|Belgorod|Izhevsk|Krasnodar|Krasnog...,,Federal State Budgetary Institution of Health ...
266,1266,NCT03540485,No,No,,Rates of neurological impairment|Rates of disa...,Individualized rates of disease progression wi...,2 years|2 years,Number of participants with treatment-related ...,To determine the incidence of adverse events a...,...,Efficacy of neuroinflammation|Axonal damage|Ox...,Neuroinflammation marker CXCL13 will be measur...,Day 0 and after 2 years|Day 0 and after 2 year...,Inclusion Criteria:||Patients who come to the ...,,No,,Seville|Seville|Seville,,Virgen Macarena Hospital|Virgen del Rocio Univ...
267,1267,NCT03564652,No,No,,Length velocity at 6 months of infant's age,Mean difference in length velocity of >0.12 cm...,"6 months of infant age, after birth and enroll...",Growth velocity at 6 months of infant's age|Le...,Mean difference in growth velocity of >0.4 gra...,...,Breast milk composition|Stool speci

In [21]:




# api_pull_2 = ct.get_study_fields(
#     search_expr="Parkinson",
#     fields=["NCTId"],
#     max_studies=10,
#     min_rnk=5,
#     fmt="csv",
# )

# # ClinicalTrials limits API queries to 1000 records
# # Count of studies may be useful to build loops when you want to retrieve more than 1000 records



# # Read the csv data in Pandas
# import pandas as pd

# temp = pd.DataFrame.from_records(api_pull_2[1:], columns=api_pull_2[0])
# temp

In [22]:




# # Get the NCTId, Condition and Brief title fields from 500 studies related to Coronavirus and Covid, in csv format.
# api_pull_1 = ct.get_study_fields(
#     search_expr="Parkinson",
#     fields=["NCTId", "Condition", "OfficialTitle", "BriefTitle" , "Acronym" , "StudyType",
#     "InterventionType","InterventionName","InterventionOtherName","InterventionDescription","Phase" 
#     ,"StudyFirstSubmitDate","LastUpdateSubmitDate","CompletionDate","OverallStatus","IsFDARegulatedDrug","IsFDARegulatedDevice","BriefSummary"],
#     max_studies=999,
#     fmt="csv",
# )

# # ClinicalTrials limits API queries to 1000 records
# # Count of studies may be useful to build loops when you want to retrieve more than 1000 records



# # Read the csv data in Pandas
# import pandas as pd

# df1=pd.DataFrame.from_records(api_pull_1[1:], columns=api_pull_1[0])

# # 
# # df1 = pd.DataFrame.to_frame().reset_index()



In [23]:
# api_pull_2 = ct.get_study_fields(
#     search_expr="Parkinson",
#     fields=["NCTId","IsFDARegulatedDrug","IsFDARegulatedDevice", "IsUnapprovedDevice", "PrimaryOutcomeMeasure", "PrimaryOutcomeDescription","PrimaryOutcomeTimeFrame", "SecondaryOutcomeMeasure","SecondaryOutcomeDescription", "SecondaryOutcomeTimeFrame","OtherOutcomeMeasure"
# ,"OtherOutcomeDescription","OtherOutcomeTimeFrame","EligibilityCriteria","StudyPopulation","HealthyVolunteers", "ReferencePMID", "LocationCity", "LocationState" , "LocationFacility"

# ],
#     max_studies=999,
#     fmt="csv",
# )

# # ClinicalTrials limits API queries to 1000 records
# # Count of studies may be useful to build loops when you want to retrieve more than 1000 records

# ct.get_study_count(search_expr="Parkinson")

# # Read the csv data in Pandas
# import pandas as pd

# df2 = pd.DataFrame.from_records(api_pull_2[1:], columns=api_pull_2[0])

##cleaning

In [24]:
# print(type(df1))

In [25]:
result = pd.merge(new_df, new_df2,  on='NCTId', how='outer')

In [26]:
result.shape

(1268, 40)

In [27]:
# result = result.loc[:,~result.columns.duplicated()]

In [28]:
# result.shape

In [29]:
# result.columns.tolist()

IMPORTANT - this just drops the second index column - monitor or watch for mismatch or if you change the number of elements that are pulled from the first api

In [30]:
result = result.drop(result.columns[[20]], axis=1)

In [31]:
result.shape

(1268, 39)

In [32]:
# result.columns.tolist()

Save file

In [33]:
# result.to_csv(r'data_raw.csv', index = False)
result.to_excel(disease+'_data_raw.xlsx', index = False)

In [34]:
#results to csv
result.to_csv(disease+'_data_raw.csv', index = False)

In [35]:
result.to_hdf(disease+'_data_raw.h5', key='df', mode='w')  

In [36]:
# labels, levels = pd.factorize(result["Phase"])  

In [37]:
# labels

In [38]:
# fun_barplot()